In [21]:
!pip install psycopg2 
#postgres baza, u anacondi je instaliraj
import psycopg2 as ps
import pandas as pd


In [22]:
df = pd.read_csv('queenpanda1') 

df.head()

,video_id,video_title,upload_date,view_count,like_count,comment_count
0,sqfZU9_4VnE,Queen The Greatest Live: We Are The Champions ...,2023-03-31,18168,2103,156
1,6SEFD609S44,Queen The Greatest Live: Tie Your Mother Down ...,2023-03-29,17890,2345,55
2,zF-5Cco1--s,Queen + Adam Lambert Return For The Rhapsody T...,2023-03-24,133884,2930,361
3,s8w-KpG5vsc,Queen The Greatest Live: Tie Your Mother Down ...,2023-03-24,48633,3028,168
4,8HmtqXLSRsE,Roger Taylor and Brian May discuss Queen&#39;s...,2023-03-22,36426,3249,62


In [23]:
#PRESLOZI FUNKCIJE

In [24]:
def connect_to_db(host, dbname, port, username, password):
    try:
        conn = ps.connect(host = host, database = dbname, user=username, password=password, port=port)
    except ps.OperationalError as e:
        raise e
    else:
        print("Connected")
        
        return conn
    
    #povezivanje s bazom

In [25]:
host = 'localhost'
dbname = 'Hellokate'
port = '5432'
username = 'XX'
password = 'XX'
conn = None
conn = connect_to_db(host,dbname, port,username, password)
curr = conn.cursor()

Connected


In [26]:
#prvo kreiranje tabele u samoj bazi aws online u koju punimo nasu pandu
def create_table(curr):
    create_table_command = ("""CREATE TABLE IF NOT EXISTS videos (
                    video_id VARCHAR(255) PRIMARY KEY,
                    video_title TEXT NOT NULL,
                    upload_date DATE NOT NULL DEFAULT CURRENT_DATE,
                    view_count INTEGER NOT NULL,
                    like_count INTEGER NOT NULL,
                    comment_count INTEGER NOT NULL
           )""")
    curr.execute(create_table_command)

In [27]:
#insert command
#currsor pod njom


def insert_into_table(curr,video_id, video_title, upload_date, view_count, like_count, comment_count): 
    insert_into_videos = ("""INSERT INTO videos(video_id, video_title, upload_date, view_count,
                        like_count, comment_count)
    VALUES(%s,%s,%s,%s,%s,%s);""")

    row_to_insert = (video_id, video_title, upload_date, view_count, like_count, comment_count)
    curr.execute(insert_into_videos, row_to_insert)
    
    #insertintovideos je kao query2

In [28]:
#sad za update 
#query i currsor pod njom 
def update_row(curr,video_id, video_title, view_count, like_count, comment_count):
    query = ("""UPDATE videos
            SET video_title = %s,
                view_count = %s,
                like_count= %s,
                comment_count= %s
            WHERE video_id = %s;""")
    vars_to_update = (video_title,view_count,like_count,comment_count,video_id)
    curr.execute(query,vars_to_update)

In [29]:
def check_if_video_exsists(curr, video_id):
    query = ("""SELECT video_id FROM VIDEOS WHERE video_id = %s""")
    curr.execute(query,(video_id,)) #provjeri ovaj zarez
    #video id as a tuple nedds to be passed
    
    return curr.fetchone() is not None
                        
#fetchone vraća 1 red, znači ako s nađe video s traženim videoid-om, vraća 1 row,ako ne nađe videoid,
#vraća none

In [30]:
# def truncate_table(curr):
#     truncate_table = ("""TRUNCATE TABLE videos""")

#     curr.execute(truncate_table)
    
    #ovo nismo imali u tutorialu provjeri

In [31]:
def append_from_df_to_db(curr,df):

    for i, row in df.iterrows():
        insert_into_table(curr, row['video_id'], row['video_title'],row['upload_date'],row['view_count'],
                          row['like_count'],row['comment_count'])
      #ovdje ide i upload jer novi red cili se uploada, a kad azuriranje nema se sta azurirat
    #datum uplaoda nego samo broj lajkova ik omenta i sl.

In [32]:
#općenito update nove baze, gore je napravili a) sad je azuriramo baz podataka aws npr. 
#ili sa update row ako već postoji taj video id ili prenosimo podatke u temp df i insert new row s podatcima
def update_db(curr,df):#napravit ćemo temporarary dataframe di provjeravat 
    temp_df = pd.DataFrame(columns=['video_id','video_title','view_count','upload_date','like_count','comment_count'])

    for i, row in df.iterrows():
        if check_if_video_exsists(curr,row['video_id']):
            update_row(curr, row['video_id'], row['video_title'], row['view_count'],row['like_count'],row['comment_count'])
        else:
            temp_df=temp_df.append(row)
    
    return temp_df

In [33]:
# host = 'localhost'
# dbname = 'Hellokate'
# port = '5432'
# username = 'XXXX'
# password = 'XXXX'
# conn = None
# conn = connect_to_db(host_name,dbname, port,username, password)
# curr = conn.cursor()

In [34]:
create_table(curr)

In [35]:
df=update_db(curr,df)
conn.commit()

In [36]:
append_from_df_to_db(curr, df)
conn.commit()

In [37]:
curr.execute("SELECT * FROM VIDEOS")
print(curr.fetchall())

[('sqfZU9_4VnE', 'Queen The Greatest Live: We Are The Champions (Episode 11)', datetime.date(2023, 3, 31), 18168, 2103, 156), ('6SEFD609S44', 'Queen The Greatest Live: Tie Your Mother Down (Episode 10) out now! #shorts #queenthegreatestlive', datetime.date(2023, 3, 29), 17890, 2345, 55), ('zF-5Cco1--s', 'Queen + Adam Lambert Return For The Rhapsody Tour Across North America!', datetime.date(2023, 3, 24), 133884, 2930, 361), ('s8w-KpG5vsc', 'Queen The Greatest Live: Tie Your Mother Down (Episode 10)', datetime.date(2023, 3, 24), 48633, 3028, 168), ('8HmtqXLSRsE', 'Roger Taylor and Brian May discuss Queen&#39;s Live Lighting Secrets 💡! #shorts #queenthegreatestlive', datetime.date(2023, 3, 22), 36426, 3249, 62)]


In [18]:
#napravili smo online tablicu za popunit je 

In [19]:
# for i, row in queenpanda.iterrows():
#     if check_if_video_exsists():
#         update_row(curr, row['video_id'], row['video_title'], row['view_count'],row['like_count'],row['comment_count'],row['video_id'])
#     else:
#         append(row)

In [20]:
#new_vid_df = update_db(curr,df) 

#OK PROVJERI JEL UPDATE QUEENPANDA ILI NOVA TABLICA #UPDATE POSTOJECA WUEEN
#A INSERT NOVA TABLICAS TEMP DF u biti prvo sa queenpanda probas cili proces i dogodi se upadte redova pa s novom
#tablicom i dogodi se insert novih redova

In [29]:
#conn.commit()